In [91]:
from src.controller.HwpController import HwpController
from collections import OrderedDict

import re, uuid
import numpy as np

In [92]:
import re, uuid
import numpy as np

from src.service.ParseService import ParseService


class ProperParser:
    def delete_non_target_data(self, table_data):
        """
        한글 표에서 원하지 않은 부분까지 나온 데이터를 정리하여 리스트로 반환합니다.
        target_data_text에 표에 반복적으로 들어가는 텍스트를 입력하여 필요없는 데이터를 삭제합니다.
        """
        target_data_text = ['일자', '계측위치', '진동레벨', '진동속도', '소음레벨']

        target_data = [
            sublist for sublist in table_data
            if any(
                entry['row'] in ['0', '1'] and any(keyword in entry['text'] for keyword in target_data_text)
                for entry in sublist
            )
        ]

        return target_data

    def extract_columns(self, table_list):
        """
        한글 표에서 공통적인 컬럼 부분을 추출합니다.
        컬럼은 대부분 표의 시작 부분에 작성되기 때문에 row 값은 0 혹은 1에 위치하게 됩니다.
        이후 컬럼 값들이 딕셔너리로 저장되어 있기 때문에 중복을 제거 후 리스트로 반환합니다.
        """
        columns = []
        for items in table_list:
            for item in items:
                if item not in [i for i in columns]:
                    if int(item['row']) == 0 or int(item['row']) == 1:
                        columns.append(item)

        return columns
    
    def extract_non_column_data(self, table_list, columns):
        """
        한글 표에서 컬럼 부분을 제거한 나머지 데이터들을 반환합니다.
        """
        return [[item for item in items[len(columns):] if not int(item['colspan']) > 1]for items in table_list]
    
    def group_by_date(self, dict_list):
        """
        한글 표 데이터를 날짜 별로 분류하여 리스트로 저장하여 반환합니다.
        """
        group_list = []

        for items in dict_list:
            rows = list(set([int(item['row']) for item in items]))
            for row in rows:
                temp = [item for item in items if int(item['row'])==row]
                group_list.append(temp)
        
        return group_list
    
    def update_merge_data(self, group_list):
        """
        한글 표에 병합 처리된 셀에 대한 데이터 처리를 완료한 뒤 리스트로 반환합니다.
        병합 처리된 셀은 rowspan 값을 이용하여 탐지한 뒤 해당 row값이 없는 리스트에 추가해 줍니다.
        """
        cached_merge_data = []
        cached_head_data = []
        
        for idx, items in enumerate(group_list):
            head_data = [data for data in items if int(data['col']) == 0 and int(data['rowspan']) > 1]
            merge_data = [data for data in items if int(data['col']) != 0 and int(data['rowspan']) > 1]
            
            if head_data:
                cached_head_data = head_data
            if merge_data:
                cached_merge_data = merge_data

            new_items = []
            if any(data not in items for data in cached_head_data):
                new_items.extend(cached_head_data)
            if any(data not in items for data in cached_merge_data):
                new_items.extend(cached_merge_data)

            group_list[idx] = new_items + items
        
        return group_list
    
    def serialize_to_dict(self, group_list, columns):
        """
        컬럼 리스트와 파싱이 끝난 그룹 리스트를 이용해서 데이터를 분류한 뒤 리스트로 반환합니다.
        컬럼 리스트에 대응하는 값들을 그룹 리스트에서 찾아서 추가해주는 작업을 수행합니다.
        """
        serialize_list = []

        columns = [item for item in columns if int(item['colspan']) <= 1]
        for items in group_list:
            data = {column['text']: item['text'] for column, item in zip(columns, items) if column != '발파횟수'}
            serialize_list.append(data)

        return serialize_list


In [129]:
service = ProperParser()
controller = HwpController()
path = '/Users/dobby/Desktop/Workspace/2월 계측보고서(호남고철5-2공구).hwp'

In [130]:
controller.get_table_list(path, '일자별 계측 현황')
table_list = controller.get_list()
table_list = service.delete_non_target_data(table_list)

unmatched field end


In [131]:
columns = service.extract_columns(table_list)

for idx, i in enumerate(columns):
    print(f'row: {i['row']}, rowspan: {i['rowspan']}, col: {i['col']}, colspan: {i['colspan']}, text: {i['text']}')
    if int(i['row']) == 0 and int(i['rowspan']) > 1 and int(i['col']) == max([int(c['col']) for c in columns]):
        columns.append(columns.pop(idx))

columns

row: 0, rowspan: 2, col: 0, colspan: 1, text: 일자
row: 0, rowspan: 2, col: 1, colspan: 1, text: 발파횟수
row: 0, rowspan: 2, col: 2, colspan: 1, text: 시간
row: 0, rowspan: 1, col: 3, colspan: 3, text: 관리기준
row: 0, rowspan: 1, col: 6, colspan: 3, text: 측정결과
row: 0, rowspan: 2, col: 9, colspan: 1, text: 계측위치
row: 1, rowspan: 1, col: 4, colspan: 1, text: 진동레벨(dB(V))
row: 1, rowspan: 1, col: 5, colspan: 1, text: 소음레벨(dB(A))
row: 1, rowspan: 1, col: 6, colspan: 1, text: 진동속도(cm/s)
row: 1, rowspan: 1, col: 7, colspan: 1, text: 진동레벨(dB(V))
row: 1, rowspan: 1, col: 8, colspan: 1, text: 소음레벨(dB(A))
row: 0, rowspan: 2, col: 9, colspan: 1, text: 계측위치


[{'row': '0', 'col': '0', 'colspan': '1', 'rowspan': '2', 'text': '일자'},
 {'row': '0', 'col': '1', 'colspan': '1', 'rowspan': '2', 'text': '발파횟수'},
 {'row': '0', 'col': '2', 'colspan': '1', 'rowspan': '2', 'text': '시간'},
 {'row': '0', 'col': '3', 'colspan': '3', 'rowspan': '1', 'text': '관리기준'},
 {'row': '0', 'col': '6', 'colspan': '3', 'rowspan': '1', 'text': '측정결과'},
 {'row': '1',
  'col': '3',
  'colspan': '1',
  'rowspan': '1',
  'text': '진동속도(cm/s)'},
 {'row': '1',
  'col': '4',
  'colspan': '1',
  'rowspan': '1',
  'text': '진동레벨(dB(V))'},
 {'row': '1',
  'col': '5',
  'colspan': '1',
  'rowspan': '1',
  'text': '소음레벨(dB(A))'},
 {'row': '1',
  'col': '6',
  'colspan': '1',
  'rowspan': '1',
  'text': '진동속도(cm/s)'},
 {'row': '1',
  'col': '7',
  'colspan': '1',
  'rowspan': '1',
  'text': '진동레벨(dB(V))'},
 {'row': '1',
  'col': '8',
  'colspan': '1',
  'rowspan': '1',
  'text': '소음레벨(dB(A))'},
 {'row': '0', 'col': '9', 'colspan': '1', 'rowspan': '2', 'text': '계측위치'}]

In [132]:
dict_list = service.extract_non_column_data(table_list, columns)

dict_list

[[{'row': '2', 'col': '0', 'colspan': '1', 'rowspan': '3', 'text': '2월1일'},
  {'row': '2', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '1회'},
  {'row': '2', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '7:04'},
  {'row': '2', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '0.2'},
  {'row': '2', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': '75'},
  {'row': '2', 'col': '5', 'colspan': '1', 'rowspan': '1', 'text': '70'},
  {'row': '2', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '2', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '2', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '2', 'col': '9', 'colspan': '1', 'rowspan': '1', 'text': '강정1리마을회관'},
  {'row': '3', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '2회'},
  {'row': '3', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '13:01'},
  {'row': '3', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '0.2'},
  {'row': '3', 'c

In [133]:
group_list = service.group_by_date(dict_list)

group_list

[[{'row': '2', 'col': '0', 'colspan': '1', 'rowspan': '3', 'text': '2월1일'},
  {'row': '2', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '1회'},
  {'row': '2', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '7:04'},
  {'row': '2', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '0.2'},
  {'row': '2', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': '75'},
  {'row': '2', 'col': '5', 'colspan': '1', 'rowspan': '1', 'text': '70'},
  {'row': '2', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '2', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '2', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '2',
   'col': '9',
   'colspan': '1',
   'rowspan': '1',
   'text': '강정1리마을회관'}],
 [{'row': '3', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '2회'},
  {'row': '3', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '13:01'},
  {'row': '3', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '0.2'},
  {'

In [134]:
group_list = service.update_merge_data(group_list)

group_list

[[{'row': '2', 'col': '0', 'colspan': '1', 'rowspan': '3', 'text': '2월1일'},
  {'row': '2', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '1회'},
  {'row': '2', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '7:04'},
  {'row': '2', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '0.2'},
  {'row': '2', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': '75'},
  {'row': '2', 'col': '5', 'colspan': '1', 'rowspan': '1', 'text': '70'},
  {'row': '2', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '2', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '2', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '2',
   'col': '9',
   'colspan': '1',
   'rowspan': '1',
   'text': '강정1리마을회관'}],
 [{'row': '2', 'col': '0', 'colspan': '1', 'rowspan': '3', 'text': '2월1일'},
  {'row': '3', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '2회'},
  {'row': '3', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '13:01'},
  {

In [135]:
serialize_list = service.serialize_to_dict(group_list, columns)

serialize_list

[{'일자': '2월1일',
  '발파횟수': '1회',
  '시간': '7:04',
  '진동속도(cm/s)': '감지안됨',
  '진동레벨(dB(V))': '감지안됨',
  '소음레벨(dB(A))': '감지안됨',
  '계측위치': '강정1리마을회관'},
 {'일자': '2월1일',
  '발파횟수': '2회',
  '시간': '13:01',
  '진동속도(cm/s)': '감지안됨',
  '진동레벨(dB(V))': '감지안됨',
  '소음레벨(dB(A))': '감지안됨',
  '계측위치': '강정2리마을회관'},
 {'일자': '2월1일',
  '발파횟수': '3회',
  '시간': '20:18',
  '진동속도(cm/s)': '감지안됨',
  '진동레벨(dB(V))': '감지안됨',
  '소음레벨(dB(A))': '감지안됨',
  '계측위치': '강정1리마을회관'},
 {'일자': '2월2일',
  '발파횟수': '1회',
  '시간': '7:01',
  '진동속도(cm/s)': '감지안됨',
  '진동레벨(dB(V))': '감지안됨',
  '소음레벨(dB(A))': '감지안됨',
  '계측위치': '강정1리마을회관'},
 {'일자': '2월2일',
  '발파횟수': '2회',
  '시간': '13:10',
  '진동속도(cm/s)': '감지안됨',
  '진동레벨(dB(V))': '감지안됨',
  '소음레벨(dB(A))': '감지안됨',
  '계측위치': '강정1리마을회관'},
 {'일자': '2월2일',
  '발파횟수': '3회',
  '시간': '19:39',
  '진동속도(cm/s)': '감지안됨',
  '진동레벨(dB(V))': '감지안됨',
  '소음레벨(dB(A))': '감지안됨',
  '계측위치': '강정2리마을회관'},
 {'일자': '2월3일',
  '발파횟수': '1회',
  '시간': '7:01',
  '진동속도(cm/s)': '감지안됨',
  '진동레벨(dB(V))': '감지안됨',
  '소음레벨(dB(A))': '감지안됨',
  '계

In [136]:
import pandas as pd

# pandas DataFrame으로 변환
df = pd.DataFrame(serialize_list)

# 계측 위치별로 데이터 프레임 분류
locations = df['계측위치'].unique()  # 계측 위치별 고유값을 찾음
print(df['계측위치'].unique())

# 각 위치별 데이터 프레임을 딕셔너리에 저장
dfs = {}
for location in locations:
    dfs[location] = df[df['계측위치'] == location]

# 결과 확인
# 예시: 과천지식정보타운 위치의 데이터 프레임 출력
print(dfs['강정1리마을회관'])

['강정1리마을회관' '강정2리마을회관' '감지안됨' '미굴발파' '지렁이농장' '성원축산' '무안공항진입도로' '태양광발전설비'
 '66.52' '65.98' '61.12' '67.72' '태양광진입도로' '미누네펜션' '청운로505-2']
       일자 발파횟수     시간 진동속도(cm/s) 진동레벨(dB(V)) 소음레벨(dB(A))      계측위치
0    2월1일   1회   7:04       감지안됨        감지안됨        감지안됨  강정1리마을회관
2    2월1일   3회  20:18       감지안됨        감지안됨        감지안됨  강정1리마을회관
3    2월2일   1회   7:01       감지안됨        감지안됨        감지안됨  강정1리마을회관
4    2월2일   2회  13:10       감지안됨        감지안됨        감지안됨  강정1리마을회관
6    2월3일   1회   7:01       감지안됨        감지안됨        감지안됨  강정1리마을회관
7    2월3일   2회  12:55       감지안됨        감지안됨        감지안됨  강정1리마을회관
8    2월3일   3회  18:28       감지안됨        감지안됨        감지안됨  강정1리마을회관
13   2월6일   2회  13:13       감지안됨        감지안됨        감지안됨  강정1리마을회관
14   2월6일   3회  19:35       감지안됨        감지안됨        감지안됨  강정1리마을회관
16   2월7일   2회  13:25       감지안됨        감지안됨        감지안됨  강정1리마을회관
17   2월7일   3회  19:25       감지안됨        감지안됨        감지안됨  강정1리마을회관
19  2월13일   1회   7:24       감지안됨        감지안됨        감지안됨  강정